In [1]:
%matplotlib inline 
import numpy as np

import arrayfire as af
from scipy import ndimage
import matplotlib.pyplot as plt
from LearnAF import *
import tqdm
import pandas as pd

af.set_backend('cpu')

DF = pd.read_csv('/home/narsi/Downloads/train.csv')
Data = np.asarray(DF, dtype = np.float32)/255.0
Data = Data[1:]
labels = np.asarray(DF['label'], dtype = np.float32)
classes = np.asarray(to_categorical(labels), dtype = np.float32)

# initialize weights randomly with mean 0
syn0 = np.array(2*np.random.random((Data.shape[1],64)) - 1, dtype = np.float32)
W1 = Variable(af.np_to_af_array(syn0),name='W1')
b1 = Variable(af.constant(0,1),name='b1')

syn0 = np.array(2*np.random.random((64,16)) - 1, dtype = np.float32)
W2 = Variable(af.np_to_af_array(syn0),name='W2')
b2 = Variable(af.constant(0,1),name='b2')

syn0 = np.array(2*np.random.random((16,10)) - 1, dtype = np.float32)
W3 = Variable(af.np_to_af_array(syn0),name='W3')
b3 = Variable(af.constant(0,1),name='b3')

w = [W1,b1,W2,b2,W3,b3]

# Model

In [2]:
def learner(X,W):
    # 784 -> 64
    X1 = sigmoid(add(matmul(X,W[0]),W[1]))
    # 64 -> 16
    X2 = sigmoid(add(matmul(X1,W[2]),W[3]))
    # 16 -> 10
    YP = sigmoid(add(matmul(X2,W[4]),W[5]))
    return YP

# Place Holders

In [3]:
Xin = Constant(af.np_to_af_array(np.zeros((32,Data.shape[1]),dtype = np.float32)))
Y = Constant(af.np_to_af_array(np.zeros((32,10),dtype = np.float32)))

In [6]:
YP = learner(Xin,w)
e = mse(Y,YP)
acc = accuracy(Y,YP)
sgd = SGD(lr = 0.01,momentum=0.99)

In [7]:
for i in range(25):
    epoch_acc = 0.
    epoch_loss = 0.
    total_batchs = int(Data.shape[0]/32)
    for j in range(total_batchs):
        X_np = Data[j*32:(j+1)*32,:]
        Xin = Constant(af.np_to_af_array(X_np))
        Y_np = classes[j*32:(j+1)*32,:]
        Y = Constant(af.np_to_af_array(Y_np))
        
        YP = learner(Xin,w)
        e = mse(Y,YP)
        acc = accuracy(Y,YP)
        (l,w) = sgd.update(e, w, i)
        epoch_acc += acc.eval()/total_batchs
        epoch_loss += l/total_batchs
        
    print('Accuracy :'+str(epoch_acc))
    print('Loss :'+str(epoch_loss))

Accuracy :0.8999999761581146
Loss :0.09167686633470402


KeyboardInterrupt: 

In [9]:

# input dataset
X = np.array([  [0,0,1],
                [0,1,1],
                [1,0,1],
                [1,1,1] ], dtype = np.float32)
X = Constant(af.np_to_af_array(X))
# output dataset            
y = np.array([[0,0,1,1]], dtype = np.float32).T
y = Constant(af.np_to_af_array(y)) 
# seed random numbers to make calculation
# deterministic (just a good practice)
np.random.seed(1)

# initialize weights randomly with mean 0
syn0 = np.array(2*np.random.random((3,10)) - 1, dtype = np.float32)
W1 = Variable(af.np_to_af_array(syn0),name='W1')
b1 = Variable(af.constant(0,1),name='b1')

syn0 = np.array(2*np.random.random((10,1)) - 1, dtype = np.float32)
W2 = Variable(af.np_to_af_array(syn0),name='W2')
b2 = Variable(af.constant(0,1),name='b2')

w = [W1,b1,W2,b2]

X1 = sigmoid(add(matmul(X,W1),b1))
yp = sigmoid(add(matmul(X1,W2),b2))
e = mse(y,yp)
acc = accuracy(y,yp)
sgd = SGD(lr = 0.001,momentum=0.99)